<a href="https://colab.research.google.com/github/EdWangLoDaSc/Dropout-as-a-Grid-Search_Representing-Model-Uncertainty-in-Deep-Learning/blob/main/AIDD_vae_%E6%95%B0%E6%8D%AE%E9%A2%84%E5%A4%84%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
from google.colab import drive
drive.mount('./content')

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).
Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [87]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as kr

from sklearn.model_selection import train_test_split

In [88]:
dataset = pd.read_csv('/content/content/MyDrive/CGAN_Regression/CGAN_code/Datasets/Datasets.csv',dtype = np.float32)


In [181]:
X = dataset.drop(['Calculated-pChEMBL'], axis = 1).values
y = dataset['Calculated-pChEMBL'].values

X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.33, random_state=42)

In [134]:
num_features = len(X_train[0])
batch_size = 128
epochs = 200
hidden_1 = 128 
hidden_2 = 64

In [60]:
#X_train = torch.tensor(X_train)
#y_train = torch.tensor(y_train)
#X_test = torch.tensor(X_test)
#y_test = torch.tensor(y_test)

#train = torch.utils.data.TensorDataset(X_train,y_train)
#test = torch.utils.data.TensorDataset(X_test,y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy 

In [91]:
print(X_train)

[[0. 0. 1. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [42]:
#train_dataloader = DataLoader(train, batch_size = 128, shuffle = True)
#test_dataloader = DataLoader(test, batch_size = 1, shuffle = False)

In [140]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [141]:
def plot_digits(X, y, encoder, batch_size=128):
    """Plots labels and MNIST digits as function of 2D latent vector

    Parameters:
    ----------
    encoder: Model
        A Keras Model instance
    X: np.ndarray
        Test data
    y: np.ndarray
        Test data labels
    batch_size: int
        Prediction batch size
    """
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(X, batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y)
    plt.colorbar()
    plt.xlabel("z[0] Latent Dimension")
    plt.ylabel("z[1] Latent Dimension")
    plt.show()

In [93]:
inputs = kr.Input(shape=(num_features, ))
encoder = kr.layers.Dense(hidden_1, activation='relu')(inputs)
encoder = kr.layers.Dense(hidden_2, activation='relu')(encoder)
encoder_model = kr.Model(inputs, encoder, name='encoder')
encoder_model.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512)]             0         
                                                                 
 dense_4 (Dense)             (None, 128)               65664     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
Total params: 73,920
Trainable params: 73,920
Non-trainable params: 0
_________________________________________________________________


In [95]:
latent_dim = kr.Input(shape=(hidden_2, ))
decoder = kr.layers.Dense(hidden_1, activation='relu')(latent_dim)
decoder = kr.layers.Dense(num_features, activation='relu')(decoder)
decoder_model = kr.Model(latent_dim, decoder, name='decoder')
decoder_model.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 512)               66048     
                                                                 
Total params: 74,368
Trainable params: 74,368
Non-trainable params: 0
_________________________________________________________________


In [96]:
outputs = decoder_model(encoder_model(inputs))
mnist_model = kr.Model(inputs, outputs )
mnist_model.compile(optimizer='adam', loss='mse')
mnist_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 512)]             0         
                                                                 
 encoder (Functional)        (None, 64)                73920     
                                                                 
 decoder (Functional)        (None, 512)               74368     
                                                                 
Total params: 148,288
Trainable params: 148,288
Non-trainable params: 0
_________________________________________________________________


In [196]:
mnist_model.fit(x=X_train, y=X_train, batch_size=100, shuffle=False, epochs=epochs)

Epoch 1/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0316
Epoch 2/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0314
Epoch 3/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0311
Epoch 4/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0312
Epoch 5/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0313
Epoch 6/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0320
Epoch 7/200
7/7 [==============================] - 0s 8ms/step - loss: 0.0326
Epoch 8/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0335
Epoch 9/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0327
Epoch 10/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0319
Epoch 11/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0313
Epoch 12/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0308
Epoch 13/200
7/7 [==============================] - 0s 6ms/st

In [102]:
def plot_predictions(y_true, y_pred):    
    f, ax = plt.subplots(2, 10, figsize=(15, 4))
    for i in range(10):
        ax[0][i].imshow(np.reshape(y_true[i])
        ax[1][i].imshow(np.reshape(y_pred[i])
    plt.tight_layout()

In [198]:

y_pred_vae = mnist_model.predict(X)
new_x = pd.DataFrame(y_pred_vae)
new_x.to_csv('3_512_vae.csv')

n = pd.read_csv('3_512_vae.csv')
print(n)

     Unnamed: 0         0         1         2         3         4         5  \
0             0  0.000000  1.177279  0.728479  0.000000  0.000000  1.108953   
1             1  0.089629  1.081324  0.000000  0.876963  0.013758  0.947484   
2             2  0.088274  0.894519  0.000000  0.980179  0.000000  1.100354   
3             3  0.000000  0.000000  0.674096  0.000000  0.000000  0.000000   
4             4  0.000000  0.000000  0.000000  0.000000  0.000000  0.887474   
..          ...       ...       ...       ...       ...       ...       ...   
926         926  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
927         927  1.057692  0.000000  0.000000  0.000000  0.000000  0.003997   
928         928  1.099436  0.000000  0.000000  0.000000  0.000000  0.000000   
929         929  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
930         930  0.903378  0.000000  0.000000  0.000000  0.000000  0.000000   

       6    7         8  ...       502  503  504   

In [149]:
def sampling(args):
    """Reparameterization trick. Instead of sampling from Q(z|X), 
    sample eps = N(0,I) z = z_mean + sqrt(var)*eps.

    Parameters:
    -----------
    args: list of Tensors
        Mean and log of variance of Q(z|X)

    Returns
    -------
    z: Tensor
        Sampled latent vector
    """

    z_mean, z_log_var = args
    eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1.0, name='epsilon')
    z = z_mean + tf.exp(z_log_var / 2) * eps
    return z


In [206]:
hidden_dim = 128
latent_dim = 10

In [207]:
inputs = kr.layers.Input(shape=(num_features, ), name='input')
x = kr.layers.Dense(hidden_dim, activation='relu')(inputs)
z_mean = kr.layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = kr.layers.Dense(latent_dim, name='z_log_var')(x)

In [208]:
z = kr.layers.Lambda(sampling, name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = kr.Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 512)]        0           []                               
                                                                                                  
 dense_36 (Dense)               (None, 128)          65664       ['input[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 10)           1290        ['dense_36[0][0]']               
                                                                                                  
 z_log_var (Dense)              (None, 10)           1290        ['dense_36[0][0]']               
                                                                                            

In [209]:
latent_inputs = kr.layers.Input(shape=(latent_dim,), name='z_sampling')
x = kr.layers.Dense(hidden_dim, activation='relu')(latent_inputs)
outputs = kr.layers.Dense(num_features, activation='relu')(x)

# instantiate decoder model
decoder = kr.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 10)]              0         
                                                                 
 dense_37 (Dense)            (None, 128)               1408      
                                                                 
 dense_38 (Dense)            (None, 512)               66048     
                                                                 
Total params: 67,456
Trainable params: 67,456
Non-trainable params: 0
_________________________________________________________________


In [210]:
outputs = decoder(encoder(inputs)[2])  # Select the Z value from outputs of the encoder
vae = kr.Model(inputs, outputs, name='vae')


In [211]:
reconstruction_loss = tf.losses.mean_squared_error(inputs, outputs)
reconstruction_loss = reconstruction_loss * num_features

# KL Divergence loss
kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

Model: "vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 512)]        0           []                               
                                                                                                  
 encoder (Functional)           [(None, 10),         68244       ['input[0][0]']                  
                                 (None, 10),                                                      
                                 (None, 10)]                                                      
                                                                                                  
 decoder (Functional)           (None, 512)          67456       ['encoder[0][2]']                
                                                                                                

In [212]:
vae.fit(X_train, epochs=1000, batch_size=256, validation_data=(X_test, None))

Epoch 1/1000
3/3 [==============================] - 1s 90ms/step - loss: 69.6533 - val_loss: 67.6013
Epoch 2/1000
3/3 [==============================] - 0s 22ms/step - loss: 65.7942 - val_loss: 64.9063
Epoch 3/1000
3/3 [==============================] - 0s 22ms/step - loss: 63.6212 - val_loss: 62.8399
Epoch 4/1000
3/3 [==============================] - 0s 26ms/step - loss: 61.9590 - val_loss: 60.5383
Epoch 5/1000
3/3 [==============================] - 0s 27ms/step - loss: 59.3056 - val_loss: 58.7741
Epoch 6/1000
3/3 [==============================] - 0s 24ms/step - loss: 58.1368 - val_loss: 57.6297
Epoch 7/1000
3/3 [==============================] - 0s 25ms/step - loss: 56.7602 - val_loss: 55.9069
Epoch 8/1000
3/3 [==============================] - 0s 24ms/step - loss: 55.0495 - val_loss: 55.0547
Epoch 9/1000
3/3 [==============================] - 0s 25ms/step - loss: 54.0289 - val_loss: 54.0802
Epoch 10/1000
3/3 [==============================] - 0s 24ms/step - loss: 53.0695 - val_los

In [213]:

y_pred_vae = vae.predict(X)
#plot_predictions(y_true, y_pred)

new_x = pd.DataFrame(y_pred_vae)
new_x.to_csv('3_512_vae_x.csv')

n = pd.read_csv('3_512_vae_x.csv')
print(n)

     Unnamed: 0         0         1         2         3         4         5  \
0             0  0.000000  0.580312  0.894221  0.359923  0.000000  1.235842   
1             1  0.207925  1.075836  0.000000  0.811016  0.000000  1.065301   
2             2  0.137868  0.926265  0.000000  1.062172  0.000000  1.028330   
3             3  0.000000  0.537253  0.445388  0.149085  0.229414  0.308316   
4             4  0.000000  0.000000  0.702448  0.054570  0.000000  1.148620   
..          ...       ...       ...       ...       ...       ...       ...   
926         926  0.707356  0.000000  0.000000  0.000000  0.000000  0.000000   
927         927  0.902292  0.025674  0.000000  0.000000  0.000000  0.000000   
928         928  0.743435  0.000000  0.000000  0.000000  0.000000  0.000000   
929         929  0.000000  0.066935  0.000000  0.000000  0.000000  0.169841   
930         930  0.666401  0.000000  0.000000  0.000000  0.000000  0.000000   

            6         7         8  ...       502   